# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Olist packages
from olist.data import Olist

In [6]:
from olist.review import Review
from olist.order import Order
from olist.product_updated import Product
from olist.seller_updated import Seller

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import nltk
import string
import text_unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup

👩🏻‍🏫 If you followed the `Decision Science` module, you already have the `olist` package installed and importable. *You can skip this section* and move to the **Data collection** section.

### (0.1) Import `olist` package`

Download a fresh version of the `olist` package:

```bash
mkdir ~/code/lewagon
cd ~/code/lewagon
git clone git@github.com:lewagon/olist.git
cd olist
git fetch
git checkout full-package
```

### (0.2) Download the datasets

- Download the datasets from Kaggle https://www.kaggle.com/olistbr/brazilian-ecommerce
- Unzip them into the `/data/csv` directory of the `olist` package:

```bash
.
├── README.md
├── data
│   └── csv
│       ├── olist_customers_dataset.csv
│       ├── olist_geolocation_dataset.csv
│       ├── olist_order_items_dataset.csv
│       ├── olist_order_payments_dataset.csv
│       ├── olist_order_reviews_dataset.csv
│       ├── olist_orders_dataset.csv
│       ├── olist_products_dataset.csv
│       ├── olist_sellers_dataset.csv
│       └── product_category_name_translation.csv
├── notebooks
├── olist
│   ├── README.md
│   ├── __init__.py
│   ├── data.py
│   ├── order.py
│   ├── product.py
│   ├── product_updated.py
│   ├── review.py
│   ├── seller.py
│   ├── seller_updated.py
│   └── utils.py
├── requirements.txt
└── setup.p
```

### (0.3) Install the `olist` package

```bash
pip install -e .
```

⚠️ Restart the kernel.

## (1) Data Collection and Preparation

👉 The way we designed the `reviews` DataFrame will help us focus on the `product categories`.

In [7]:
reviews = Review().get_training_data()
reviews.head()

,review_id,length_review,review_score,order_id,product_category_name
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer


👉 We can retrieve the reviews from `order_reviews` within the Olist class.

In [8]:
data = Olist().get_data()

In [9]:
reviews_data = data['order_reviews']
reviews_data.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


🪤 We need to collect some information about the orders because:
- Back then, customers could review an order even before receiving it
- They could also rate an order that they hadn't received

In [10]:
orders = data['orders']
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


💥 Let's merge these three datasets and apply some operations

In [16]:
# YOUR CODE HERE
df = reviews.merge(reviews_data,on='order_id',how='inner').merge(orders,on='order_id',how='inner')
# .join(orders,on='order_id',how='inner')

🚓 Remove the reviews that were written even before receiving the order.

In [24]:
# YOUR CODE HERE
df['review_creation_date'] = pd.to_datetime(df['review_creation_date'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])


🚓 Remove the reviews for undelivered orders.

In [29]:
# YOUR CODE HERE
df2 = df[df['review_creation_date'] >= df['order_delivered_customer_date']].copy()
df2.head()

,review_id_x,length_review,review_score_x,order_id,product_category_name,review_id_y,review_score_y,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,7bc2406110b926393aa56f80a40eba40,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,80e641a11e56f04c1ad469d5645fdfde,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,228ce5500dc1d8e020d8d1322874b6f0,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,e64fb393e7b32834bb789ff8bb30750e,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,f7c4243c7fe1938f181bec41a392bdeb,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


✍️ As some people fill in only the title or the body of a review, it may be a good idea to aggregate them together.



<details>
    <summary>💡<i>Hint</i></summary>

* Drop rows with both a missing title and a missing comment
* Refer to the documentation [pandas.DataFrame.dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) and have a look at the `how` argument...

In [31]:
# YOUR CODE HERE
df2['review_comment_title'] = df2['review_comment_title'].fillna('')
df2['review_comment_message'] = df2['review_comment_message'].fillna('')
df2['reviews'] = df2['review_comment_title'] + df2['review_comment_message']
df2.head()

,review_id_x,length_review,review_score_x,order_id,product_category_name,review_id_y,review_score_y,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,reviews
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,7bc2406110b926393aa56f80a40eba40,4,,,2018-01-18,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,80e641a11e56f04c1ad469d5645fdfde,5,,,2018-03-10,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00,
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,228ce5500dc1d8e020d8d1322874b6f0,5,,,2018-02-17,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00,
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,e64fb393e7b32834bb789ff8bb30750e,5,,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,f7c4243c7fe1938f181bec41a392bdeb,5,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00,Parabéns lojas lannister adorei comprar pela I...


🕵🏻 Let's focus on some columns of interest:

In [34]:
# Focus on certain columns
columns_of_interest = ['review_id_x',
                       'length_review',
                       'review_score_x',
                       'order_id',
                       'product_category_name',
                       'reviews']
df2 = df2[columns_of_interest]
df2.head()

,review_id_x,length_review,review_score_x,order_id,product_category_name,reviews
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela I...


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews. **Beware NLTK has no Portuguese lemmatizer** (often but they do have a stemmer `nltk.stem.RSLPStemmer`. 

In [35]:
# YOUR CODE HERE
from nltk.stem import RSLPStemmer
def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
#     stop_words = set(stopwords.words('english')) ## define stopwords
    
#     tokenized_sentence_cleaned = [ ## remove stopwords
#         w for w in tokenized_sentence if not w in stop_words
#     ]

    lemmatized = [
        RSLPStemmer().stem(word) 
        for word in tokenized_sentence
    ]
    

    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [37]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /Users/jinru/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [38]:
# YOUR CODE HERE
df2['reviews'] = df2['reviews'].apply(cleaning)

In [39]:
df2.head()

,review_id_x,length_review,review_score_x,order_id,product_category_name,reviews
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,receb bem ant do praz estipul
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,parabém loj lannist ador compr pel internet se...


## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [40]:
# YOUR CODE HERE
df2['review_score_x'].value_counts()

5    55347
4    18172
3     7350
1     6043
2     2425
Name: review_score_x, dtype: int64

🕵🏻‍♂️ Let's focus on these reviews...

In [42]:
# YOUR CODE HERE
df3 = df2[df2['review_score_x'] <= 3]
df3.head()

,review_id_x,length_review,review_score_x,order_id,product_category_name,reviews
5,15197aa66ff4d0650b5434f1b46cda19,0,1,b18dcdf73be66366873cd26c5724d1dc,cama_mesa_banho,
14,9a0abbb668bafb95a6d2b05db43284c4,0,3,d7bd0e4afdf94846eb73642b4e3e75c3,pet_shop,
18,fdbdb2629a7cde0f66657acc92084e7f,0,3,70a752414a13d09cc1f2b437b914b28e,bebes,
45,ab3056e4fb5a36df4b58c1746e4a6b8f,0,2,aad1dcbe4c9fe2e3486e5e04c6649097,relogios_presentes,
56,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,receb soment control mide split estil falt con...


### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [43]:
# YOUR CODE HERE
vectorizer = TfidfVectorizer(max_df=0.75,ngram_range = (2,2),max_features=5000)
vectorized_documents = vectorizer.fit_transform(df3['reviews'])
vectorized_documents = pd.DataFrame(vectorized_documents.toarray(), 
                                    columns = vectorizer.get_feature_names_out())

vectorized_documents

abaix da  abaix do  abert com  abr caix  abr embal  abr fech  \
0           0.0       0.0        0.0       0.0        0.0       0.0   
1           0.0       0.0        0.0       0.0        0.0       0.0   
2           0.0       0.0        0.0       0.0        0.0       0.0   
3           0.0       0.0        0.0       0.0        0.0       0.0   
4           0.0       0.0        0.0       0.0        0.0       0.0   
...         ...       ...        ...       ...        ...       ...   
15813       0.0       0.0        0.0       0.0        0.0       0.0   
15814       0.0       0.0        0.0       0.0        0.0       0.0   
15815       0.0       0.0        0.0       0.0        0.0       0.0   
15816       0.0       0.0        0.0       0.0        0.0       0.0   
15817       0.0       0.0        0.0       0.0        0.0       0.0   

       abr reclam  abr uma  absurd par  acab com  acab da  acab de  acab deix  \
0             0.0      0.0         0.0       0.0      0.0      0.0        0.0   
1             0.0      0.0         0.0       0.0      0.0      0.0        0.0   
2             0.0      0.0         0.0       0.0      0.0      0.0        0.0   
3             0.0      0.0         0.0       0.0      0.0      0.0        0.0   
4             0.0      0.0         0.0       0.0      0.0      0.0        0.0   
...           ...      ...         ...       ...      ...      ...        ...   
15813         0.0      0.0         0.0       0.0      0.0      0.0        0.0   
15814         0.0      0.0         0.0       0.0      0.0      0.0        0.0   
15815         0.0      0.0         0.0       0.0      0.0      0.0        0.0   
15816         0.0      0.0         0.0       0.0      0.0      0.0        0.0   
15817         0.0      0.0         0.0       0.0      0.0      0.0        0.0   

       acab em  acab fic  acab mas  acab muit  acab não  acab receb  \
0          0.0       0.0       0.0        0.0       0.0         0.0   
1          0.0       0.0       0.0        0.0       0.0         0.0   
2          0.0       0.0       0.0        0.0       0.0         0.0   
3          0.0       0.0       0.0        0.0       0.0         0.0   
4          0.0       0.0       0.0        0.0       0.0         0.0   
...        ...       ...       ...        ...       ...         ...   
15813      0.0       0.0       0.0        0.0       0.0         0.0   
15814      0.0       0.0       0.0        0.0       0.0         0.0   
15815      0.0       0.0       0.0        0.0       0.0         0.0   
15816      0.0       0.0       0.0        0.0       0.0         0.0   
15817      0.0       0.0       0.0        0.0       0.0         0.0   

       acab ruim  ach estranh  ach foss  ach mant  ach mater  ach muit  \
0            0.0          0.0       0.0       0.0        0.0       0.0   
1            0.0          0.0       0.0       0.0        0.0       0.0   
2            0.0          0.0       0.0       0.0        0.0       0.0   
3            0.0          0.0       0.0       0.0        0.0       0.0   
4            0.0          0.0       0.0       0.0        0.0       0.0   
...          ...          ...       ...       ...        ...       ...   
15813        0.0          0.0       0.0       0.0        0.0       0.0   
15814        0.0          0.0       0.0       0.0        0.0       0.0   
15815        0.0          0.0       0.0       0.0        0.0       0.0   
15816        0.0          0.0       0.0       0.0        0.0       0.0   
15817        0.0          0.0       0.0       0.0        0.0       0.0   

       ach pequen  ach praz  ach produt  ach que  ach um  acompanh produt  \
0             0.0       0.0         0.0      0.0     0.0              0.0   
1             0.0       0.0         0.0      0.0     0.0              0.0   
2             0.0       0.0         0.0      0.0     0.0              0.0   
3             0.0       0.0         0.0      0.0     0.0              0.0   
4             0.0       0.0         0.0      0.0     0

### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [44]:
# YOUR CODE HERE
# Instantiate the LDA 
n_components = 3
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_documents)

LatentDirichletAllocation(max_iter=100, n_components=3)

#### Document Mixture (of Topics)

In [45]:
# YOUR CODE HERE
document_topic_mixture = lda_model.transform(vectorized_documents)
pd.DataFrame(document_topic_mixture)

,0,1,2
0,0.333333,0.333333,0.333333
1,0.333333,0.333333,0.333333
2,0.333333,0.333333,0.333333
3,0.333333,0.333333,0.333333
4,0.309571,0.592854,0.097575
...,...,...,...
15813,0.138512,0.722965,0.138523
15814,0.533562,0.066987,0.399450
15815,0.542104,0.224267,0.233629
15816,0.333333,0.333333,0.333333


👉 Let's report the most important topic for each review

In [ ]:
# YOUR CODE HERE

#### Topic Mixture (of Words)

In [61]:
# YOUR CODE HERE
topic_mixture = \
        lda_model.components_
#         columns = vectorizer.get_feature_names_out()
    

In [47]:
topic_mixture

abaix da  abaix do  abert com  abr caix  abr embal  abr fech  abr reclam  \
0  0.347344  0.352422   0.342374  1.786091   0.358924  0.339100    4.750249   
1  2.003555  0.728998   1.711814  1.672881   4.336299  0.399116    0.343291   
2  0.346581  2.146822   0.499444  0.387644   0.384368  2.070594    0.361385   

    abr uma  absurd par  acab com   acab da   acab de  acab deix   acab em  \
0  3.771931    0.371759  0.362570  0.334388  0.351081   0.334376  0.338365   
1  0.406932    0.334628  2.063374  2.799366  0.361273   3.355519  0.365449   
2  0.361098    2.052625  0.343403  0.353975  1.786647   0.339678  1.589730   

   acab fic  acab mas  acab muit  acab não  acab receb  acab ruim  \
0  0.338418  0.363639   0.334438  0.334714    2.186864   0.334782   
1  0.787758  1.715461   2.293832  2.590057    0.361539   3.594339   
2  1.608591  0.368536   0.337061  0.373098    0.387209   0.341460   

   ach estranh  ach foss  ach mant  ach mater  ach muit  ach pequen  ach praz  \
0     0.334339  0.414082  0.334406   0.334629  0.350925    0.334366  2.346369   
1     1.856445  3.399895  2.820699   2.604016  8.494996    3.234910  1.095571   
2     0.356342  0.337215  0.334518   0.348964  0.366990    0.334500  0.351460   

   ach produt    ach que    ach um  acompanh produt  acontec com  acontec iss  \
0    0.336071   0.364866  1.859541         0.362499     1.496450     1.000003   
1    6.383249  30.883852  3.598684         1.627338     0.374090     0.341871   
2    0.342664   5.030925  0.369574         1.317501     1.873776     2.172105   

   acontec que  acord com  acredit que   ades de  adquir não  adquir produt  \
0     0.360250   0.350624     0.351693  1.149152    0.351156       1.024020   
1     0.350608   0.357725     0.401965  2.169212    1.979872       2.121639   
2     3.226567   9.467825    10.642881  0.343363    1.399640       0.394051   

   adquir um  agenc do  agor com  agor est   agor nad  agor nao  agor nem  \
0   0.335842  2.234167  3.495807  3.166209  11.841149  4.085885  1.281036   
1   2.099591  0.339000  0.338444  0.341198   0.344609  0.354064  0.354966   
2   0.358398  0.351071  0.406394  0.359441   0.341071  0.361620  0.911976   

   agor ninguém   agor não  agor que  agor receb   agor só  agor tenh  \
0      3.076490  19.665755  2.293091    2.820115  4.264365   1.974161   
1      0.334628   0.340219  0.360960    0.585977  0.336180   0.343974   
2      0.339459   0.363778  0.976382    0.354062  0.353053   1.155521   

   aguard contat  aguard da  aguard de  aguard do  aguard encomend  \
0       7.111860   6.685114   5.353297   3.291413         1.485437   
1       0.351411   0.337125   0.353143   0.347007         0.334045   
2       0.363266   0.347197   0.343087   0.345013         0.334022   

   aguard entreg  aguard envi  aguard orient  aguard os  aguard outr  \
0      14.495698     3.632001       0.357030   2.150713     4.285385   
1       0.336566     0.334896       2.889577   0.335812     0.335009   
2       0.341778     0.349515       0.343642   0.353227     0.337353   

   aguard par  aguard pel  aguard pos  aguard posicion  aguard provid  \
0    4.790705    2.079361    2.163178         0.500049       3.277037   
1    0.408648    0.338240    0.334752         0.335804       0.335805   
2    0.346911    0.340338    0.335197         1.954144       0.408972   

   aguard receb  aguard resoluç  aguard respost  aguard retorn  aguard soluç  \
0      4.314805        1.997511       11.139152      18.400129      9.596501   
1      0.335261        0.363060        0.358419       0.366355      0.341343   
2      0.340936        1.220697        0.410585       0.363691      0.348297   

   aguard troc  aguard um  aguard uma  agênc do  aind aguard  aind assim  \
0     0.355753   5.317909    7.613062  5.294049     5.620901    0.553129   
1     0.353448   0.336891    0.336884  0.342742     0.344896    0.345882   
2     4.521839   0.357136    0.343318  0.339080     0.351873    1.846566   

   aind com  aind entreg   aind est  a

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [48]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [49]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))
        

🕵🏻‍♂️ Print the topics with their topwords:

In [51]:
n_components = topic_mixture.shape[0]
n_components

3

In [59]:
topic_mixture

RangeIndex(start=0, stop=3, step=1)

In [63]:
print_topics(vectorizer, lda_model, 10)

--------------------
Topic 0:
[('foi entreg', 95.41), ('não receb', 73.35), ('receb apen', 67.88), ('só receb', 63.15), ('muit bom', 56.5), ('est aguard', 55.59), ('not fiscal', 54.74), ('receb um', 48.84), ('do produt', 48.52), ('compr doi', 43.53)]
--------------------
Topic 1:
[('receb produt', 73.79), ('do produt', 67.29), ('não receb', 65.63), ('não gost', 62.67), ('difer do', 47.95), ('nao receb', 42.82), ('um pouc', 41.95), ('do que', 41.92), ('não recom', 39.82), ('gost do', 39.6)]
--------------------
Topic 2:
[('vei com', 88.02), ('produt não', 83.4), ('com defeit', 81.68), ('do praz', 78.19), ('produt vei', 70.5), ('produt cheg', 65.53), ('no praz', 61.01), ('foi entreg', 48.81), ('ant do', 48.19), ('dentr do', 46.66)]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _produto = product_
- _recomendo = recommend (não recomendo == not recommend)_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _errado = wrong_
- _gostaria = I would like to..._
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [ ]:
# YOUR CODE HERE

In [64]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

KeyError: 'most_important_topic'

In [ ]:
df[["review_id", 
        "review_score", 
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head()

## (3.4) Pipeline Tf-Idf and LDA

In [ ]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [ ]:
# YOUR CODE HERE

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [ ]:
pipeline._final_estimator

In [ ]:
pipeline._final_estimator.components_

**Document Mixture** with the Pipeline:

In [ ]:
# YOUR CODE HERE

**Topic Mixture** with the Pipeline:

In [ ]:
# YOUR CODE HERE

## (4) 🎁 Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [ ]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'), 
                                                          ('review_score', 'std')], 
                                                    ascending = [False, True])
product_categories

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [ ]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [ ]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id", 
                        "review_score", 
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [ ]:
worst_products_reviews["most_important_topic"].value_counts()

In [ ]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

In [ ]:
[topic_word_mixture[i] for i in bad_frequency]

## (5) 🎁 Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [ ]:
sellers = Seller().get_training_data()
sellers.columns

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [ ]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits", 
    ascending = True).head(10)
worst_sellers

### (5.2) Products sold by the worst sellers

In [ ]:
products = Product().get_training_data() [["product_id", "category"]]
products

❓ What are the types of products sold by the worst sellers ❓

In [ ]:
sellers_product_category = data["order_items"].merge(products, 
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

In [ ]:
sellers_product_category.groupby("seller_id").count()

### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [ ]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [ ]:
bad_reviews_sellers = worst_products_reviews.merge(data["order_items"])
bad_reviews_sellers.head(3)

In [ ]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    if len(temp) > 0: # if seller appears in the bad reviews dataframe
        most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
        return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [ ]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))
    

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`